# Audio Scene Classification

This notebook guides you through building and training a simple neural network to classify audio scenes. The task is to identify the environment or setting where an audio recording was captured, such as a 'bus', 'cafe', or 'park', based solely on the sounds present in the audio data. You will implement your own classification network to perform this task.


# Import Required Libraries

In [ ]:
import torch
import torch.nn as nn
import random
from IPython.display import Audio, display
from torchsummary import summary
from torch.utils.data import DataLoader
import os
from pathlib import Path
from torch.utils.data import random_split
import importlib
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Import the Audio Data


In [ ]:
# Download the dataset
zip_file_path = "/home/TCD/Lab-06/dataset.zip"
if not os.path.exists(zip_file_path):
    !curl --create-dirs -o {zip_file_path} https://tcddeeplearning.blob.core.windows.net/deeplearning202324/audiosceneclassification2025.zip
else:
    print(f"File already exists: {zip_file_path}")

# Unzip the dataset if the directory doesn't exist
dataset_dir = "/home/TCD/Lab-06/dataset/"
if not os.path.exists(dataset_dir):
    !unzip /home/TCD/Lab-06/dataset.zip -d {dataset_dir}
else:
    print(f"Dataset directory already exists: {dataset_dir}")

# Lets Listen to the Data 👂

The lab is composed of 15 possible scenes. The cell below plays a random clip from the training data and shows its label.

Have a listen with headphones, replay the cell to load another random clip.

In [ ]:
data_root = "/home/TCD/Lab-06/dataset/small_set_complete/"
meta_filename = "meta.txt"
audio_subdir = "audio"

root = Path(data_root).resolve()
audio_dir = root / audio_subdir
meta_path = root / meta_filename

with meta_path.open() as fh:
    meta = {}
    for line in fh:
        rel_path, label, clip_id = line.strip().split("\t")
        meta[rel_path] = {"label": label, "clip_id": clip_id}

choices = list(audio_dir.glob("*.wav"))

chosen = random.choice(choices)
rel_from_audio_dir = chosen.relative_to(audio_dir)
meta_key = (Path("audio") / rel_from_audio_dir).as_posix()
info = meta.get(meta_key) or meta.get(chosen.relative_to(root).as_posix())

print(f"Playing: {chosen}")
if info:
    print(f"Label: {info['label']} | Clip: {info['clip_id']}")
else:
    print("No metadata found for this file.")

display(Audio(filename=str(chosen)))

# Mount your drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/4c16-labs/code/lab-06/

# Tasks
1. Implement the Dataset Class in `audio_dataset.py`
2. Implement your audio classification model in `models.py`
3. Save your model to the root of the directory as `model.pth`. [*DO NOT CHANGE THE NAME*]

We have provided instructions in each file.

Do not change the file or class names.

In [ ]:
# Run this cell to reload the LoadAudio or AudioClassifier if you have modified after the first attempt
import models, audio_dataset          # import the modules themselves
importlib.reload(models)
importlib.reload(audio_dataset)
 
from models import AudioClassifier    # now import the names you need
from audio_dataset import LoadAudio

# Unit Test of Dataset and Model
In the cell below, we check your dataset and model inputs, outputs and variables are as we expect them.

If you fail any of these tests, your model or dataset will be rejected by the testing server.

In [ ]:
from function_test import test_dataset, print_model_summary, test_model_forward

# Immutable dataset args for evaluation
data_root = "/home/TCD/Lab-06/dataset"
meta_filename = "meta.txt"
audio_subdir = "audio"

# 1) Dataset tests
ds_ok, dataset, loader = test_dataset(
    LoadAudioClass=LoadAudio,
    root_dir=data_root,
    meta_filename=meta_filename,
    audio_subdir=audio_subdir,
    torch=torch,
    DataLoader=DataLoader,
    batch_size=1
)

# 2) Model summary & param budget
num_classes = getattr(dataset, "num_classes", 15)
model = AudioClassifier(num_classes=num_classes).to(device)
ms_ok = print_model_summary(model, summary, feature_example_shape=(1, 220500), max_params=5_000_000)

# 3) Model forward tests
mf_ok = test_model_forward(model, loader, torch, device=device, num_classes=num_classes)

print("\n=== Overall ===")
print("ALL TESTS PASSED ✅" if (ds_ok and ms_ok and mf_ok) else "SOME TESTS FAILED ❌")


# Train your model
Call your dataset, optimizers, scheduelers, implement training and validation dataloaders. Write your training loop.



## Data Preprocessing and Feature Engineering
*   **Spectrograms:** Convert audio signals into spectrograms (time-frequency representations). These can be treated as images and processed using Convolutional Neural Networks (CNNs) designed for 2D data.

*   **Mel-Scale Frequency Cepstral Coefficients (MFCCs):** Instead of using raw audio waveforms, consider extracting more informative features like MFCCs. The human ear perceives frequencies on a non-linear scale (mel scale). MFCCs capture the spectral envelope of the sound, which is more relevant for human perception and often leads to better performance in audio tasks.
